In [1]:
from scipy.signal import butter, lfilter
import pandas as pd
import numpy as np
import csv
import glob
import matplotlib.pyplot as plt
import datetime
from zipfile import ZipFile

In [2]:
def get_utc_date_time(ts):
    return pd.to_datetime(ts, unit='s', infer_datetime_format = True, utc = True)
#.strftime('%H:%M:%S:%f')

def add_fs(sample_rate, date):
    return date + datetime.timedelta(milliseconds=1.0/(sample_rate) * 1000.0)

In [3]:
def time_range(df_length, start_timestamp, sample_rate):
    # Generate date time range based on sample rate and starting timestamp
    time_range = []
    t_0 = get_utc_date_time(float(start_timestamp))
    time_range.append(t_0)
    
    next_date = t_0
    
    for count in range(df_length-1):
        next_date = add_fs(sample_rate, next_date)
        time_range.append(next_date)
    return time_range

In [4]:
def format_time(t):
    s = t.strftime('%Y-%m-%d %H:%M:%S.%f')
    return s[:-3]

In [5]:
base_path = r"C:\Users\shaye\Documents\EAT Lab\Sensor_Data\Empatica data"
participant_id = "PR025"
base_path

'C:\\Users\\shaye\\Documents\\EAT Lab\\Sensor_Data\\Empatica data'

In [6]:
list_of_zips = glob.glob(base_path + '\A*.zip')

In [7]:
eda_all = []

for zip_file in list_of_zips:
    zf = ZipFile(zip_file)
    eda_df_raw = pd.read_csv(zf.open('EDA.csv'))
    
    starting_timestamp = eda_df_raw.columns[0]
    sample_rate = eda_df_raw[starting_timestamp][0]
    
    eda_df_raw = eda_df_raw.drop([0, 0])
    eda_df = eda_df_raw.copy()
    
    time_col = time_range(len(eda_df_raw), starting_timestamp, sample_rate)
    eda_df['Timestamp'] = time_col
    eda_df['Timestamp'] = eda_df.apply(lambda x: format_time(x['Timestamp']) + '+00:00', axis=1)
    
    eda_df = eda_df.rename(columns={starting_timestamp:"EDA"})
    eda_df['ParticipantId'] = participant_id
    
    eda_all.append(eda_df)

In [8]:
eda_df_all = pd.concat(eda_all)

In [9]:
eda_df_all

,EDA,Timestamp,ParticipantId
1,0.000000,2020-12-15 13:59:08.000+00:00,PR025
2,0.838918,2020-12-15 13:59:08.250+00:00,PR025
3,1.062043,2020-12-15 13:59:08.500+00:00,PR025
4,0.996791,2020-12-15 13:59:08.750+00:00,PR025
5,0.999353,2020-12-15 13:59:09.000+00:00,PR025
...,...,...,...
158792,2.158767,2021-01-14 01:34:41.750+00:00,PR025
158793,2.116501,2021-01-14 01:34:42.000+00:00,PR025
158794,2.074235,2021-01-14 01:34:42.250+00:00,PR025
158795,2.049900,2021-01-14 01:34:42.500+00:00,PR025
